In [3]:
# https://medium.com/huggingface/100-times-faster-natural-language-processing-in-python-ee32033bdced
%load_ext Cython

In [8]:
import cProfile
import pstats
# import my_slow_module

cProfile.run('my_slow_module.run()', 'restats')
p = pstats.Stats('restats')
p.sort_stats('cumulative').print_stats(30)
print('hello world!')

NameError: name 'my_slow_module' is not defined

In [1]:
from random import random

class Rectangle:
    def __init__(self, w, h):
        self.w = w
        self.h = h
    def area(self):
        return self.w * self.h

def check_rectangles_py(rectangles, threshold):
    n_out = 0
    for rectangle in rectangles:
        if rectangle.area() > threshold:
            n_out += 1
    return n_out

def main_rectangles_slow():
    n_rectangles = 10000000
    rectangles = list(Rectangle(random(), random()) for i in range(n_rectangles))
    n_out = check_rectangles_py(rectangles, threshold=0.25)
    print(n_out)

In [2]:
%%time
# Let's run it:
main_rectangles_slow()

4032847
CPU times: user 6.12 s, sys: 390 ms, total: 6.51 s
Wall time: 6.51 s


In [6]:
%%cython -a

from cymem.cymem cimport Pool
from random import random

cdef struct Rectangle:
    float w
    float h

cdef int check_rectangles_cy(Rectangle* rectangles, int n_rectangles, float threshold):
    cdef int n_out = 0
    # C arrays contain no size information => we need to state it explicitly
    for rectangle in rectangles[:n_rectangles]:
        if rectangle.w * rectangle.h > threshold:
            n_out += 1
    return n_out

def main_rectangles_fast():
    cdef int n_rectangles = 10000000
    cdef float threshold = 0.25
    cdef Pool mem = Pool()
    cdef Rectangle* rectangles = <Rectangle*>mem.alloc(n_rectangles, sizeof(Rectangle))
    for i in range(n_rectangles):
        rectangles[i].w = random()
        rectangles[i].h = random()
    n_out = check_rectangles_cy(rectangles, n_rectangles, threshold)
    print(n_out)

In [5]:
%%time
main_rectangles_fast()

4032690
CPU times: user 252 ms, sys: 20 ms, total: 273 ms
Wall time: 272 ms
